In [18]:
import scanpy as sc
import tempfile
import subprocess
import os

In [2]:
data_dir = 'data/'

In [3]:
adata = sc.read(data_dir+'processed/04seurat_integrated_noprior.h5ad')

In [4]:
sc.pp.neighbors(adata)
sc.tl.louvain(adata, resolution=0.5, key_added='louvain_post')

         Falling back to preprocessing with `sc.pp.pca` and default params.


## Trying code

In [ ]:
group1 = 'louvain'
group2 = 'louvain_post'

In [ ]:
def write_tmp_labels(adata, group_name='louvain'):
    
    label_map = {}
    i = 1
    for label in adata.obs[group_name].unique():
        label_map[label] = i
        i += 1
    labels = '\n'.join([str(label_map[name]) for name in adata.obs[group_name]])
    
    f = tempfile.NamedTemporaryFile(delete=False)
    f.write(str.encode(labels))
    f.close()
    
    return f.name

In [69]:
group1_file = write_tmp_labels(adata, group1)
group2_file = write_tmp_labels(adata, group2)

In [95]:
onmi_dir = "../../Overlapping-NMI/"
nmi_call = subprocess.Popen(
    [onmi_dir+"onmi", group1_file, group2_file], 
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT)
stdout, stderr = nmi_call.communicate()
print(stderr)
nmi_out = stdout.decode()
nmi_out

None


'NMI<Max>:\t0.998732\nOther measures:\n  lfkNMI:\t0.998732\n  NMI<Sum>:\t0.998732\n'

In [92]:
nmi_split = [x.strip().split('\t') for x in nmi_out.split('\n')]

In [94]:
nmi_max = nmi_split[0][1]
nmi_max

'0.998732'

## Finished Function

In [15]:
def write_tmp_labels(adata, group_name):
    label_map = {}
    i = 1
    for label in adata.obs[group_name].unique():
        label_map[label] = i
        i += 1
    labels = '\n'.join([str(label_map[name]) for name in adata.obs[group_name]])
    with tempfile.NamedTemporaryFile(delete=False) as f:
        f.write(str.encode(labels))
        filename = f.name
    return filename

In [19]:
def nmi(adata, group1, group2, onmi_dir="../../Overlapping-NMI/"):
    group1_file = write_tmp_labels(adata, group1)
    group2_file = write_tmp_labels(adata, group2)
    
    nmi_call = subprocess.Popen(
        [onmi_dir+"onmi", group1_file, group2_file], 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT)
    
    stdout, stderr = nmi_call.communicate()
    print(stderr)
    nmi_out = stdout.decode()
    print(nmi_out)
    
    nmi_split = [x.strip().split('\t') for x in nmi_out.split('\n')]
    nmi_max = nmi_split[0][1]
    
    # remove temporary files
    os.remove(group1_file)
    os.remove(group2_file)
    return nmi_max

In [20]:
nmi(adata, group1='louvain', group2='louvain_post', onmi_dir="../../Overlapping-NMI/")

None
NMI<Max>:	0.998732
Other measures:
  lfkNMI:	0.998732
  NMI<Sum>:	0.998732



'0.998732'